# Week 3 Assignment of Applied Data Science Capstone

## Task 1: Transform the data on Wiki page into pandas dataframe

Import necessary library

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

Grep the wiki page content by using BeautifulSoup

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
wiki_html = requests.get(url).text
soup = BeautifulSoup(wiki_html, 'html.parser')

Convert content of PostalCode HTML table as list of data

In [3]:
data = []
for tr in soup.tbody.find_all('tr'):
    data.append([ td.get_text().strip() for td in tr.find_all('td')])

Create the dataframe

In [4]:
df = pd.DataFrame(data, columns=['PostalCode','Borough','Neighborhood'])

In [5]:
df.head()

,PostalCode,Borough,Neighborhood
0,None,None,None
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village


In [6]:
df.shape

(290, 3)

Remove 'Not assigned' rows

In [7]:
not_assigned = 'Not assigned'
not_assigned_row = df[ (df.Borough == not_assigned) & (df.Neighborhood == not_assigned) ]
not_assigned_row.head(), not_assigned_row.shape

(   PostalCode       Borough  Neighborhood
 1         M1A  Not assigned  Not assigned
 2         M2A  Not assigned  Not assigned
 10        M8A  Not assigned  Not assigned
 14        M2B  Not assigned  Not assigned
 21        M7B  Not assigned  Not assigned, (77, 3))

In [8]:
df.drop(not_assigned_row.index, inplace=True)

Remove all 'None' rows

In [9]:
df.dropna(inplace=True)
df.head()

,PostalCode,Borough,Neighborhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights


In [10]:
df.shape

(212, 3)

Create Neighborhood column

In [11]:
def neighborhood_list(grouped):    
    if( len(grouped) == 1 ):
        # only one line under the postal code assign the Borough as Neighborhood
        borough = grouped['Borough'].tolist()[0] 
        neighborhood = grouped['Neighborhood'].tolist()[0] 
        if( neighborhood) == not_assigned:
            return borough
        else:
            return neighborhood
    else:
        # transform grouped Neighborhood as single value separated with commas
        return ', '.join(sorted(grouped['Neighborhood'].tolist())) 
                    
grp = df.groupby(['PostalCode', 'Borough'])
df2 = grp.apply(neighborhood_list).reset_index(name='Neighborhood')

In [12]:
df2[df2.PostalCode == 'M7A']

,PostalCode,Borough,Neighborhood
85,M7A,Queen's Park,Queen's Park


In [13]:
df2

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Highland Creek, Port Union, Rouge Hill"
2,M1E,Scarborough,"Guildwood]], Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [14]:
df2.shape

(103, 3)

## Task 2: Add the latitude and longitude coordinates to the dataframe

Install geocoder library

In [15]:
!pip install geocoder
import geocoder

distributed 1.21.8 requires msgpack, which is not installed.


Define a function to retrieve latitude and longitude

In [18]:
def get_latlng(postal_code):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
    return lat_lng_coords
    
get_latlng('M2H')

[43.80284500000005, -79.35623615099996]

Retrieve coordinates of all neighborhoods by Postal Code

In [19]:
postal_codes = df2['PostalCode']    
coords = [ get_latlng(postal_code) for postal_code in postal_codes.tolist() ]

Add Latitude and Longitude information to dataframe

In [20]:
df2_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])
df2['Latitude'] = df2_coords['Latitude']
df2['Longitude'] = df2_coords['Longitude']

In [21]:
df2.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.811650,-79.195561
1,M1C,Scarborough,"Highland Creek, Port Union, Rouge Hill",43.785605,-79.158701
2,M1E,Scarborough,"Guildwood]], Morningside, West Hill",43.765690,-79.175299
3,M1G,Scarborough,Woburn,43.768216,-79.217610
4,M1H,Scarborough,Cedarbrae,43.769608,-79.239440


In [22]:
df2[df2.PostalCode == 'M5G']

,PostalCode,Borough,Neighborhood,Latitude,Longitude
57,M5G,Downtown Toronto,Central Bay Street,43.656091,-79.38493


In [23]:
df2

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.811650,-79.195561
1,M1C,Scarborough,"Highland Creek, Port Union, Rouge Hill",43.785605,-79.158701
2,M1E,Scarborough,"Guildwood]], Morningside, West Hill",43.765690,-79.175299
3,M1G,Scarborough,Woburn,43.768216,-79.217610
4,M1H,Scarborough,Cedarbrae,43.769608,-79.239440
5,M1J,Scarborough,Scarborough Village,43.743085,-79.232172
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.726260,-79.263670
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.713213,-79.284910
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.723575,-79.234976
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.696690,-79.260069


## Task 3: Explore and cluster the neighborhoods in Torondo

Import required library

In [24]:
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

Solving environment: done

# All requested packages already installed.



In [25]:
toronto_coords = get_latlng('')
toronto_coords

[43.648690000000045, -79.38543999999996]

Follow instruction of assignment, only work with boroughs that contain the word Toronto

In [26]:
df3 = df2[ df2.Borough.str.contains('Toronto') ]

In [27]:
df3

,PostalCode,Borough,Neighborhood,Latitude,Longitude
37,M4E,East Toronto,The Beaches,43.676531,-79.295410
41,M4K,East Toronto,"Riverdale, The Danforth West",43.683262,-79.355120
42,M4L,East Toronto,"India Bazaar, The Beaches West",43.667985,-79.314642
43,M4M,East Toronto,Studio District,43.662766,-79.334830
44,M4N,Central Toronto,Lawrence Park,43.728135,-79.387090
45,M4P,Central Toronto,Davisville North,43.712755,-79.388514
46,M4R,Central Toronto,North Toronto West,43.714523,-79.406960
47,M4S,Central Toronto,Davisville,43.702765,-79.385769
48,M4T,Central Toronto,"Moore Park, Summerhill East",43.690505,-79.382973
49,M4V,Central Toronto,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",43.686003,-79.402335


Plot each data on the map

In [28]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[toronto_coords[0], toronto_coords[1]], zoom_start=12)

# add markers to map
for lat, lng, borough, neighborhood in zip(df3['Latitude'], df3['Longitude'], df3['Borough'], df3['Neighborhood']):
    label = '{} - {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Load FourSquare account information from JSON file

In [29]:
import json # library to handle JSON files
fs_act = None
with open('fs_act.json') as json_data:
    fs_act = json.load(json_data)

#### Let's explore the first neighborhood in our dataframe. Replicate same analysis as New York City data

Get the neighborhood's name.

In [30]:
toronto_data = df3
toronto_data.reset_index(inplace=True)

In [31]:
toronto_data.loc[0, 'Neighborhood']

'The Beaches'

Get the neighborhood's latitude and longitude values.

In [32]:
neighborhood_latitude = toronto_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = toronto_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = toronto_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of The Beaches are 43.67653131600008, -79.29540999999995.


#### Now, let's get the top 100 venues that are in East Toronto within a radius of 500 meters.

First, let's create the GET request URL. Name your URL **url**.

In [34]:
# type your answer here
VERSION = '20180605'
CLIENT_ID = fs_act['client_id']
CLIENT_SECRET = fs_act['client_secret']
latitude = neighborhood_latitude
longitude = neighborhood_longitude
radius = 500
LIMIT = 100

url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION,radius, LIMIT)


Send the GET request and examine the resutls

In [35]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ba4ee5d1ed21976f9e8b0bc'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'The Beaches',
  'headerFullLocation': 'The Beaches, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 3,
  'suggestedBounds': {'ne': {'lat': 43.68103132050008,
    'lng': -79.2891996964911},
   'sw': {'lat': 43.67203131150008, 'lng': -79.3016203035088}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4e77e3861f6ecf8d3648300c',
       'name': 'Starbucks',
       'location': {'address': '637 Kingston Rd.',
        'crossStreet': 'at Main St.',
        'lat': 43.67879837444001,
        'lng': -79.2980449760153,
  

From the Foursquare lab in the previous module, we know that all the information is in the *items* key. Before we proceed, let's borrow the **get_category_type** function from the Foursquare lab.

In [36]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now we are ready to clean the json and structure it into a *pandas* dataframe.

In [37]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Starbucks,Coffee Shop,43.678798,-79.298045
1,Grover Pub and Grub,Pub,43.679181,-79.297215
2,Upper Beaches,Neighborhood,43.680563,-79.292869


And how many venues were returned by Foursquare?

In [38]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

3 venues were returned by Foursquare.


<a id='item2'></a>

#### Explore Neighborhoods in Toronto

#### Let's create a function to repeat the same process to all the neighborhoods in Toronto

In [39]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Now write the code to run the above function on each neighborhood and create a new dataframe called *toronto_venues*.

In [40]:
toronto_venues = getNearbyVenues(toronto_data['Neighborhood'], toronto_data['Latitude'], toronto_data['Longitude'])

The Beaches
Riverdale, The Danforth West
India Bazaar, The Beaches West
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park, Summerhill East
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
Rosedale
Cabbagetown, St. James Town
Church and Wellesley
Harbourfront, Regent Park
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Adelaide, King, Richmond
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North, Forest Hill West
North Midtown, The Annex, Yorkville
Harbord, University of Toronto
Chinatown, Grange Park, Kensington Market
Bathurst Quay, CN Tower, Harbourfront West, Island airport, King and Spadina, Railway Lands, South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place, Underground city
Christie
Dovercourt Village, Dufferin
Little Portugal, Trinity
Brockton, Exhibition Place, Parkdale Village
High Park, The 

In [41]:
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676531,-79.29541,Starbucks,43.678798,-79.298045,Coffee Shop
1,The Beaches,43.676531,-79.29541,Grover Pub and Grub,43.679181,-79.297215,Pub
2,The Beaches,43.676531,-79.29541,Upper Beaches,43.680563,-79.292869,Neighborhood
3,"Riverdale, The Danforth West",43.683262,-79.35512,Dairy Queen,43.684223,-79.357062,Ice Cream Shop
4,"Riverdale, The Danforth West",43.683262,-79.35512,Dollarama,43.686300,-79.355893,Discount Store


In [42]:
toronto_venues.shape

(1718, 7)

#### Let's check the size of the resulting dataframe

In [43]:
print(toronto_venues.shape)
toronto_venues.head()

(1718, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676531,-79.29541,Starbucks,43.678798,-79.298045,Coffee Shop
1,The Beaches,43.676531,-79.29541,Grover Pub and Grub,43.679181,-79.297215,Pub
2,The Beaches,43.676531,-79.29541,Upper Beaches,43.680563,-79.292869,Neighborhood
3,"Riverdale, The Danforth West",43.683262,-79.35512,Dairy Queen,43.684223,-79.357062,Ice Cream Shop
4,"Riverdale, The Danforth West",43.683262,-79.35512,Dollarama,43.686300,-79.355893,Discount Store


Let's check how many venues were returned for each neighborhood

In [44]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
"Bathurst Quay, CN Tower, Harbourfront West, Island airport, King and Spadina, Railway Lands, South Niagara",66,66,66,66,66,66
Berczy Park,61,61,61,61,61,61
"Brockton, Exhibition Place, Parkdale Village",54,54,54,54,54,54
Business reply mail Processing Centre969 Eastern,100,100,100,100,100,100
"Cabbagetown, St. James Town",43,43,43,43,43,43
Central Bay Street,97,97,97,97,97,97
"Chinatown, Grange Park, Kensington Market",100,100,100,100,100,100
Christie,10,10,10,10,10,10


#### Let's find out how many unique categories can be curated from all the returned venues

In [45]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 210 uniques categories.


<a id='item3'></a>

### Analyze Each Neighborhood

In [46]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Adult Boutique,Afghan Restaurant,American Restaurant,Antique Shop,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,...,Theater,Thrift / Vintage Store,Toy / Game Store,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


And let's examine the new dataframe size.

In [47]:
toronto_onehot.shape

(1718, 210)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [48]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Adult Boutique,Afghan Restaurant,American Restaurant,Antique Shop,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,...,Theater,Thrift / Vintage Store,Toy / Game Store,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint
0,"Adelaide, King, Richmond",0.000000,0.0000,0.0000,0.040000,0.000000,0.00,0.010000,0.000000,0.000000,...,0.010000,0.000000,0.000000,0.00,0.010000,0.000000,0.000000,0.010000,0.000000,0.000000
1,"Bathurst Quay, CN Tower, Harbourfront West, Is...",0.015152,0.0000,0.0000,0.015152,0.000000,0.00,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,Berczy Park,0.000000,0.0000,0.0000,0.000000,0.000000,0.00,0.016393,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,"Brockton, Exhibition Place, Parkdale Village",0.000000,0.0000,0.0000,0.000000,0.000000,0.00,0.000000,0.000000,0.018519,...,0.018519,0.018519,0.000000,0.00,0.037037,0.000000,0.000000,0.000000,0.000000,0.000000
4,Business reply mail Processing Centre969 Eastern,0.000000,0.0000,0.0000,0.040000,0.000000,0.00,0.010000,0.000000,0.000000,...,0.020000,0.000000,0.000000,0.00,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000
5,"Cabbagetown, St. James Town",0.000000,0.0000,0.0000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,Central Bay Street,0.000000,0.0000,0.0000,0.020619,0.000000,0.00,0.010309,0.010309,0.000000,...,0.010309,0.000000,0.010309,0.00,0.010309,0.010309,0.010309,0.010309,0.000000,0.000000
7,"Chinatown, Grange Park, Kensington Market",0.000000,0.0000,0.0000,0.000000,0.000000,0.01,0.020000,0.000000,0.010000,...,0.000000,0.000000,0.010000,0.00,0.050000,0.000000,0.040000,0.010000,0.000000,0.000000
8,Christie,0.000000,0.0000,0.0000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,Church and Wellesley,0.000000,0.0125,0.0125,0.012500,0.000000,0.00,0.000000,0.000000,0.012500,...,0.012500,0.000000,0.000000,0.00,0.012500,0.012500,0.012500,0.000000,0.000000,0.012500


#### Let's confirm the new size

In [49]:
toronto_grouped.shape

(38, 210)

#### Let's print each neighborhood along with the top 5 most common venues

In [50]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
                 venue  freq
0          Coffee Shop  0.09
1                Hotel  0.07
2                 Café  0.06
3           Steakhouse  0.04
4  American Restaurant  0.04


----Bathurst Quay, CN Tower, Harbourfront West, Island airport, King and Spadina, Railway Lands, South Niagara----
                venue  freq
0  Italian Restaurant  0.08
1         Coffee Shop  0.08
2          Restaurant  0.06
3                 Bar  0.06
4                Café  0.05


----Berczy Park----
          venue  freq
0   Coffee Shop  0.08
1    Restaurant  0.05
2  Cocktail Bar  0.05
3   Cheese Shop  0.03
4        Bakery  0.03


----Brockton, Exhibition Place, Parkdale Village----
                    venue  freq
0             Coffee Shop  0.11
1              Restaurant  0.06
2  Furniture / Home Store  0.06
3      Italian Restaurant  0.04
4                     Bar  0.04


----Business reply mail Processing Centre969 Eastern----
                 venue  freq
0          Coffee S

#### Let's put that into a *pandas* dataframe

First, let's write a function to sort the venues in descending order.

In [51]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [52]:
import numpy as np
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Hotel,Café,Steakhouse,American Restaurant,Deli / Bodega,Gastropub,Restaurant,Burger Joint,Bakery
1,"Bathurst Quay, CN Tower, Harbourfront West, Is...",Italian Restaurant,Coffee Shop,Restaurant,Bar,Pub,Café,Park,Sandwich Place,Gym / Fitness Center,Speakeasy
2,Berczy Park,Coffee Shop,Cocktail Bar,Restaurant,Farmers Market,Beer Bar,Steakhouse,Café,Hotel,Bakery,Cheese Shop
3,"Brockton, Exhibition Place, Parkdale Village",Coffee Shop,Furniture / Home Store,Restaurant,Hotel,Gift Shop,Gym,Bar,Vegetarian / Vegan Restaurant,Sandwich Place,Café
4,Business reply mail Processing Centre969 Eastern,Coffee Shop,Restaurant,Hotel,American Restaurant,Café,Bar,Steakhouse,Japanese Restaurant,Pub,Italian Restaurant
5,"Cabbagetown, St. James Town",Coffee Shop,Restaurant,Italian Restaurant,Pizza Place,Indian Restaurant,Market,Café,Bakery,Liquor Store,Jewelry Store
6,Central Bay Street,Coffee Shop,Clothing Store,Cosmetics Shop,Café,Italian Restaurant,Tea Room,Chinese Restaurant,Plaza,Pizza Place,Thai Restaurant
7,"Chinatown, Grange Park, Kensington Market",Café,Chinese Restaurant,Vegetarian / Vegan Restaurant,Bar,Vietnamese Restaurant,Bakery,Mexican Restaurant,Dumpling Restaurant,Dim Sum Restaurant,Comfort Food Restaurant
8,Christie,Café,Grocery Store,Playground,Coffee Shop,Baby Store,Athletics & Sports,Italian Restaurant,Exhibit,Dumpling Restaurant,Event Space
9,Church and Wellesley,Coffee Shop,Japanese Restaurant,Gay Bar,Sushi Restaurant,Restaurant,Bubble Tea Shop,Pub,Burger Joint,Men's Store,Dance Studio


<a id='item4'></a>

#### Cluster Neighborhoods

Run *k*-means to cluster the neighborhood into 5 clusters.

In [53]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [54]:
len(kmeans.labels_)

38

In [55]:
toronto_grouped_clustering

,Yoga Studio,Adult Boutique,Afghan Restaurant,American Restaurant,Antique Shop,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,...,Theater,Thrift / Vintage Store,Toy / Game Store,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint
0,0.000000,0.0000,0.0000,0.040000,0.000000,0.00,0.010000,0.000000,0.000000,0.030000,...,0.010000,0.000000,0.000000,0.00,0.010000,0.000000,0.000000,0.010000,0.000000,0.000000
1,0.015152,0.0000,0.0000,0.015152,0.000000,0.00,0.000000,0.000000,0.000000,0.015152,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.0000,0.0000,0.000000,0.000000,0.00,0.016393,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.0000,0.0000,0.000000,0.000000,0.00,0.000000,0.000000,0.018519,0.000000,...,0.018519,0.018519,0.000000,0.00,0.037037,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.0000,0.0000,0.040000,0.000000,0.00,0.010000,0.000000,0.000000,0.020000,...,0.020000,0.000000,0.000000,0.00,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000
5,0.000000,0.0000,0.0000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,0.000000,0.0000,0.0000,0.020619,0.000000,0.00,0.010309,0.010309,0.000000,0.000000,...,0.010309,0.000000,0.010309,0.00,0.010309,0.010309,0.010309,0.010309,0.000000,0.000000
7,0.000000,0.0000,0.0000,0.000000,0.000000,0.01,0.020000,0.000000,0.010000,0.000000,...,0.000000,0.000000,0.010000,0.00,0.050000,0.000000,0.040000,0.010000,0.000000,0.000000
8,0.000000,0.0000,0.0000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,0.000000,0.0125,0.0125,0.012500,0.000000,0.00,0.000000,0.000000,0.012500,0.000000,...,0.012500,0.000000,0.000000,0.00,0.012500,0.012500,0.012500,0.000000,0.000000,0.012500


In [56]:
toronto_grouped['Cluster Labels'] = kmeans.labels_

In [57]:
results = toronto_grouped[['Neighborhood', 'Cluster Labels']]

In [58]:
results.head()

,Neighborhood,Cluster Labels
0,"Adelaide, King, Richmond",2
1,"Bathurst Quay, CN Tower, Harbourfront West, Is...",2
2,Berczy Park,2
3,"Brockton, Exhibition Place, Parkdale Village",2
4,Business reply mail Processing Centre969 Eastern,2


In [59]:
toronto_merged = toronto_data

# add clustering labels
toronto_merged = pd.merge(toronto_merged, results, on=['Neighborhood']).drop('index', axis=1)

In [60]:
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676531,-79.295410,2,Pub,Coffee Shop,Wings Joint,Dumpling Restaurant,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Exhibit,Event Space
1,M4K,East Toronto,"Riverdale, The Danforth West",43.683262,-79.355120,1,Park,Bus Line,Grocery Store,Ice Cream Shop,Discount Store,Wings Joint,Eastern European Restaurant,Farmers Market,Farm,Falafel Restaurant
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.667985,-79.314642,1,Park,Fish & Chips Shop,Pub,Fast Food Restaurant,Liquor Store,Light Rail Station,Sandwich Place,Board Shop,Steakhouse,Sushi Restaurant
3,M4M,East Toronto,Studio District,43.662766,-79.334830,2,Diner,Italian Restaurant,Café,Gastropub,American Restaurant,Pizza Place,Coffee Shop,Bar,Sushi Restaurant,Arts & Crafts Store
4,M4N,Central Toronto,Lawrence Park,43.728135,-79.387090,0,Bus Line,Swim School,Wings Joint,Electronics Store,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Exhibit


Finally, let's visualize the resulting clusters

In [61]:
# create map
map_clusters = folium.Map(location=[toronto_coords[0], toronto_coords[1]], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<a id='item5'></a>

#### Examine Clusters

#### Cluster 1

In [62]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Central Toronto,0,Bus Line,Swim School,Wings Joint,Electronics Store,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Exhibit


#### Cluster 2

In [63]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,East Toronto,1,Park,Bus Line,Grocery Store,Ice Cream Shop,Discount Store,Wings Joint,Eastern European Restaurant,Farmers Market,Farm,Falafel Restaurant
2,East Toronto,1,Park,Fish & Chips Shop,Pub,Fast Food Restaurant,Liquor Store,Light Rail Station,Sandwich Place,Board Shop,Steakhouse,Sushi Restaurant
5,Central Toronto,1,Breakfast Spot,Hotel,Food & Drink Shop,Burger Joint,Japanese Restaurant,Park,Department Store,Elementary School,Dance Studio,Fast Food Restaurant
6,Central Toronto,1,Park,Gym Pool,Playground,Garden,Wings Joint,Donut Shop,Farm,Falafel Restaurant,Exhibit,Event Space
8,Central Toronto,1,Park,Gym,Tennis Court,Intersection,Wings Joint,Dumpling Restaurant,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant
10,Downtown Toronto,1,Park,Building,Playground,Tennis Court,Bank,Dumpling Restaurant,Farmers Market,Farm,Falafel Restaurant,Exhibit
19,Downtown Toronto,1,Park,Harbor / Marina,Athletics & Sports,Music Venue,Wings Joint,Dumpling Restaurant,Farmers Market,Farm,Falafel Restaurant,Exhibit
23,Central Toronto,1,Park,Business Service,Wings Joint,Eastern European Restaurant,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Exhibit,Event Space
31,West Toronto,1,Park,Pet Store,Bakery,Smoke Shop,Fast Food Restaurant,Liquor Store,Café,Middle Eastern Restaurant,Supermarket,Bus Line


#### Cluster 3

In [64]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East Toronto,2,Pub,Coffee Shop,Wings Joint,Dumpling Restaurant,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Exhibit,Event Space
3,East Toronto,2,Diner,Italian Restaurant,Café,Gastropub,American Restaurant,Pizza Place,Coffee Shop,Bar,Sushi Restaurant,Arts & Crafts Store
7,Central Toronto,2,Coffee Shop,Dessert Shop,Seafood Restaurant,Sandwich Place,Café,Italian Restaurant,Pizza Place,Optical Shop,Farmers Market,Fast Food Restaurant
9,Central Toronto,2,Light Rail Station,Coffee Shop,Convenience Store,Supermarket,Eastern European Restaurant,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Exhibit
11,Downtown Toronto,2,Coffee Shop,Restaurant,Italian Restaurant,Pizza Place,Indian Restaurant,Market,Café,Bakery,Liquor Store,Jewelry Store
12,Downtown Toronto,2,Coffee Shop,Japanese Restaurant,Gay Bar,Sushi Restaurant,Restaurant,Bubble Tea Shop,Pub,Burger Joint,Men's Store,Dance Studio
13,Downtown Toronto,2,Coffee Shop,Breakfast Spot,Bakery,Park,Pub,Mexican Restaurant,Electronics Store,Restaurant,Café,Event Space
14,Downtown Toronto,2,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Japanese Restaurant,Bar,Italian Restaurant,Tea Room,Diner,Movie Theater
15,Downtown Toronto,2,Restaurant,Coffee Shop,Café,Hotel,Bakery,Gastropub,Japanese Restaurant,Seafood Restaurant,Cocktail Bar,Cosmetics Shop
16,Downtown Toronto,2,Coffee Shop,Cocktail Bar,Restaurant,Farmers Market,Beer Bar,Steakhouse,Café,Hotel,Bakery,Cheese Shop


#### Cluster 4

In [65]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
34,West Toronto,3,Convenience Store,Park,Wings Joint,Dumpling Restaurant,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Exhibit,Event Space


#### Cluster 5

In [66]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,Central Toronto,4,Health & Beauty Service,Wings Joint,Dumpling Restaurant,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Exhibit,Event Space,Ethiopian Restaurant
